# การวิเคราะห์ข้อมูลในระบบคลาวด์: วิธีการใช้ "Azure ML SDK"

## บทนำ

ในโน้ตบุ๊กนี้ เราจะเรียนรู้วิธีการใช้ Azure ML SDK เพื่อฝึกอบรม, ปรับใช้ และใช้งานโมเดลผ่าน Azure ML

ข้อกำหนดเบื้องต้น:
1. คุณได้สร้าง Azure ML workspace แล้ว
2. คุณได้โหลด [ชุดข้อมูล Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) เข้าไปใน Azure ML
3. คุณได้อัปโหลดโน้ตบุ๊กนี้เข้าไปใน Azure ML Studio

ขั้นตอนต่อไปคือ:

1. สร้าง Experiment ใน Workspace ที่มีอยู่
2. สร้าง Compute cluster
3. โหลดชุดข้อมูล
4. กำหนดค่า AutoML โดยใช้ AutoMLConfig
5. รัน AutoML experiment
6. สำรวจผลลัพธ์และเลือกโมเดลที่ดีที่สุด
7. ลงทะเบียนโมเดลที่ดีที่สุด
8. ปรับใช้โมเดลที่ดีที่สุด
9. ใช้งาน endpoint

## การนำเข้าเฉพาะของ Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## เริ่มต้นพื้นที่ทำงาน  
เริ่มต้นวัตถุพื้นที่ทำงานจากการกำหนดค่าที่บันทึกไว้ ตรวจสอบให้แน่ใจว่าไฟล์การกำหนดค่าอยู่ที่ .\config.json  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## สร้างการทดลอง Azure ML

มาสร้างการทดลองชื่อ 'aml-experiment' ใน workspace ที่เราเพิ่งตั้งค่าเสร็จกันเถอะ


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## สร้าง Compute Cluster
คุณจำเป็นต้องสร้าง [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) สำหรับการรัน AutoML ของคุณ


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ข้อมูล
ตรวจสอบให้แน่ใจว่าคุณได้อัปโหลดชุดข้อมูลไปยัง Azure ML แล้ว และคีย์มีชื่อเดียวกันกับชุดข้อมูล


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## ปรับใช้โมเดลที่ดีที่สุด

รันโค้ดด้านล่างเพื่อปรับใช้โมเดลที่ดีที่สุด คุณสามารถดูสถานะของการปรับใช้ได้ใน Azure ML portal ขั้นตอนนี้อาจใช้เวลาสักครู่


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## การใช้งาน Endpoint
คุณสามารถเพิ่มข้อมูลอินพุตลงในตัวอย่างอินพุตด้านล่างนี้ได้


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
